This script takes N (N=10 for paper) trained models, and 
evaluates N trained models on A, B, C HFO testing data. <br>
Note that model_outputs_script was used for generating model outputs in paper (this notebook was used for checking purposes).

In [32]:
import matplotlib
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.activations import relu
from tensorflow.keras import Input
from tensorflow.keras import Model
from numpy import save
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.signal import detrend
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import confusion_matrix
import pandas as pd

In [33]:
def detrend_func(X):
    
    X_raw = X[:, :, 0]
    X_raw_detrend = detrend(X_raw)
    X[:, :, 0] = X_raw_detrend
    return X

In [34]:
def load_pred(model_path, base_path, data_path):
    
    # load data 
    X = np.squeeze(np.load(base_path + data_path))
    X = detrend_func(X)
    
    y_int = np.ones(X.shape[0])

    
    # display confusion matrix
    y_pred_int = np.argmax(y_pred,axis=-1)
    cmat = confusion_matrix(y_int, y_pred_int)
    return cmat

In [48]:
def test_models(model_path, data_path, detectors):
    
    # list where length is equal to number of hfo sets (3 for paper)
    X_arr = []
    acc_list = np.zeros((3,10))

    # load data 
    for d in detectors:
        
        X = detrend_func(np.squeeze(np.load(data_path + 'X_' + d + '_test.npy')))
        y = np.load(data_path + 'y_' + d + '_test.npy')
        
        hfos = np.argwhere(y==1)
        X_arr.append(np.squeeze(X[hfos]))
         
    # loop through N = 10 models
    for j, file in enumerate(np.sort(os.listdir(model_path))):
        if file[0] == 'U':
            continue
        print(file)
        model = tf.keras.models.load_model(model_path + file + "/best_model.h5")
        
        for i, hfo_set in enumerate(X_arr):
            y_pred = model.predict(hfo_set)
            y_pred_int = np.argmax(y_pred,axis=-1)
            acc_list[i, j] = np.count_nonzero(y_pred_int) / y_pred_int.shape[0]
            print(acc_list[i,j])
            
    return acc_list
            
    
            

In [49]:
data_path = '/home3/ebrahim/ripple_code/yasa_labels/updated_data/509/'
model_path = '/home3/ebrahim/results/updated_results/a/509/'
detectors = ['a', 'b', 'c']
acc_list = test_models(model_path, data_path, detectors)

509_mm_1
0.9259135098893732
0.8528842431281456
0.659784283513097
509_mm_10
0.9386523633925579
0.867595818815331
0.6776579352850539
509_mm_2
0.9091518605430774
0.8466898954703833
0.6567026194144838
509_mm_3
0.9265839758632249
0.8478513356562137
0.6560862865947612
509_mm_4
0.9245725779416695
0.8552071234998064
0.6650231124807396
509_mm_5
0.93027153871941
0.8521099496709252
0.6607087827426811
509_mm_6
0.9228964130070398
0.8517228029423152
0.6631741140215717
509_mm_7
0.9128394233992625
0.8443670150987224
0.6551617873651772
509_mm_8
0.9024472008045592
0.8385598141695703
0.6462249614791987
509_mm_9
0.9037881327522628
0.8385598141695703
0.6434514637904468


In [55]:
np.save('/home3/ebrahim/results/updated_results/saved_acc/acc_list_509_a', acc_list)